# Lesson 3: Pre-processing Part 1 - Global Thresholds

Fluorescence datasets ideally reflect a relationship between the pixels in an image and the location and local density of your fluorescent molecule in a sample. However, properties of the detectors, optics, or even the samples can confound direct interpretation of this data. Here we will present some operations that can mitigate these effects to achieve robust hypothesis testing. 

In this module, you will
- Review how to load digital images and associated metadata
- Understand what it means to find a threshold for an image
- Use a threshold to make a mask, i.e. separate the background and foreground of an image
- Learn how to use Otsu's method to find threshold values
- Understand the mechanics of Otsu's method, and learn its limitations

We will work with data from an experiment to determine whether treatment with a drug causes a shift in the localization of a protein.

Hypothesis: Treatment with drug A will cause a decrease in the total amount of protein Y. You have saved the control dataset as "DMSO.tif" and the drugged cell dataset as "drugA.tif". 

First some boilerplate code to make it easier to access useful libraries, and to make it easier to visualize data in the notebook...

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.ndimage

In [ ]:
sns.set_style('dark', rc={'image.cmap':'inferno'})

Import an image file and associated metadata as we learned in the last lesson.

In [ ]:
from skimage.io import imread

data_drug = imread("../data/confocal_drug_panel/drugA.tif")
data_nodrug = imread("../data/confocal_drug_panel/DMSO.tif")

In [ ]:
import json
with open('../data/confocal_drug_panel/DMSO_metadata.json', mode='r') as f_nodrug:
    meta_nodrug = json.load(f_nodrug)

In [ ]:
for key, value in meta_nodrug.items():
    print(key)

In [ ]:
drug_slice = {}
nodrug_slice = {}
for idx, channel in enumerate(meta_nodrug['channels']):
    drug_slice[channel] = data_drug[3,:,:,idx]
    nodrug_slice[channel] = data_nodrug[3,:,:,idx] #add in the indexing when read in full dataset
    print(channel)

Display the images to make sure everything worked as expected...

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(16, 4))
ax[0].imshow(nodrug_slice["actin"])
ax[1].imshow(nodrug_slice['nucleus'])
ax[2].imshow(nodrug_slice["your_fav_protein"])

fig, ax = plt.subplots(1, 3, figsize=(16, 4))
ax[0].imshow(drug_slice["actin"])
ax[1].imshow(drug_slice['nucleus'])
ax[2].imshow(drug_slice["your_fav_protein"])

**Exercise** Use the metadata, the "shape" command, and/or visualization of the slice to determine what "VARIABLE" in the following cell denotes. Choose an appropriate value of VARIABLE for this dataset. Then run the cell to organize the nucleus data into a form that's easier to index 

In [ ]:
# use VARIABLE = 4 is recommended; VARIABLE = time or z-section?
VARIABLE = 1 # change me! 
nodrug_slice = {}
for idx, channel in enumerate(meta_nodrug['channels']):
    nodrug_slice[channel] = data_nodrug[VARIABLE,:,:,idx]
nucleus_data = nodrug_slice['nucleus']

In [ ]:
data_nodrug.shape

In [ ]:
# View few slices to determine VARIABLE in the previous cell
fig, ax = plt.subplots(1, 3, figsize=(16, 4))
ax[0].imshow(data_nodrug[0, :, :, 1])
ax[1].imshow(data_nodrug[4, :, :, 1])
ax[2].imshow(data_nodrug[8, :, :, 1])

ax[0].set_title("VARIABLE = 0")
ax[1].set_title("VARIABLE = 4")
ax[2].set_title("VARIABLE = 8")

Change the crop parameters to crop out a nucleus (1) in the middle of the image and (2) one on the edge of the image for visualization in the rest of the exercise.

In [ ]:
# parameters to adjust
# use minX1 = 450, minY1 = 250, minX2 = 0, minY2 = 250 is recommended
nuc_minX1 = 1 # crop edges for a cell in the center of field of view; change me!
nuc_minY1 = 1 # change me!
nuc_minX2 = 1 # crop edges for cell at the edge of the field of view; change me!
nuc_minY2 = 1 # change me!
nuc_crop_size = 200 #pix
nuc_image_view_thresh = 0.7 #set for the entire notebook so can compare outputs more easily

# run
nuc_maxX1 = nuc_minX1 + nuc_crop_size
nuc_maxY1 = nuc_minY1 + nuc_crop_size
nuc_maxX2 = nuc_minX2 + nuc_crop_size
nuc_maxY2 = nuc_minY2 + nuc_crop_size

nuc_c1 = nucleus_data[nuc_minY1 : nuc_maxY1, nuc_minX1 : nuc_maxX1] #crop of a nucleus in the center of the image
nuc_c2 = nucleus_data[nuc_minY2 : nuc_maxY2, nuc_minX2 : nuc_maxX2] #crop of the nucleus at the edge of the image

nuc_top = nucleus_data.max() * nuc_image_view_thresh

fig, ax = plt.subplots(1, 3, figsize=(16, 4))
ax[0].imshow(nucleus_data, vmin=0, vmax=nuc_top, interpolation = 'nearest')
ax[1].imshow(nuc_c1, vmin=0, vmax=nuc_top, interpolation = 'nearest')
ax[2].imshow(nuc_c2, vmin=0, vmax=nuc_top, interpolation = 'nearest')

## Image pre-processing: a motivating example

So here we have images of fixed cells in three channels -- cell bodies labeled with an actin stain, nuclei labeled with DAPI, and a third protein "your_fav_protein" that responds to drug treatment. Just by looking at the images it seems like the protein is shifting from the nuclei to the cell body once the drug is applied (always visualize your intermediates!), but it is unclear if the drug treatment changes the total amount of protein per cell as well.

To address these questions, you will need to do the following --

1. Make a mask of the actin channel to identify pixels within the cell bodies
2. Make a mask of the nuclear channel to identify pixels within the nucleus
3. Determine the signal coming from *your favorite protein* within these regions of interest. 

Today, we will define the ROIs in the image. We'll find that the quality of masks can be improved by preprocessing the images by quantitatively determining thresholds and filtering to remove noise. We will then cover how to clean up the mask and turn it into an accurate ROI using morphological image processing. 

Tomorrow, we will cover how to design your image processing pipeline to deal with some trickier problems, such as quantifying fluorescence in the ROIs determined here. 

**Preprocessing misteps are a good way to get a paper retracted. We argue that it's easier to make these misteps when doing things manually, but it's not *impossible* to do it computationally. In fact if you don't check intermediate steps of your data in either case, it's no good. Always visualize your intermediates!**

### Making masks to localize cell bodies

In [ ]:
nodrug_slice['actin'].dtype

In [ ]:
data = nodrug_slice['actin']

We need to find an appropriate threshold that defines the cell bodies accurately across the image using the sliding bar. First, view the image more closely.

In [ ]:
#parameters to adjust
minX1 = 450 #crop edges for a cell in the center of field of view
minY1 = 250
minX2 = 0 #crop edges for cell at the edge of the field of view
minY2 = 250
crop_size = 200 #pix
image_view_thresh = 0.7

#run
maxX1 = minX1 + crop_size
maxY1 = minY1 + crop_size
maxX2 = minX2 + crop_size
maxY2 = minY2 + crop_size

top = data.max() * image_view_thresh

fig, ax = plt.subplots(1, 3, figsize=(16, 4))
ax[1].imshow(data[minY1 : maxY1 , minX1 : maxX1], vmin=0, vmax=top)
ax[0].imshow(data, vmin=0, vmax=top)
ax[2].imshow(data[minY2 : maxY2, minX2: maxX2], vmin=0, vmax=top)

Determine which threshold gives the best mask across the image using the sliding bar.

In [ ]:
from ipywidgets import interactive
@interactive
def show_masks(thresh=(0, data.max() * 0.3, 40)):
    fig, ax = plt.subplots(1, 3, figsize=(16, 4))
    mask = np.zeros(nodrug_slice["actin"].shape)
    mask[nodrug_slice["actin"] >=thresh] = 1
    mask_zoom_center = mask[minY1 : maxY1 , minX1 : maxX1]
    mask_zoom_edge = mask[minY2 : maxY2 , minX2 : maxX2]
    ax[0].imshow(mask, vmin=0, vmax=1)
    ax[1].imshow(mask_zoom_center, vmin=0, vmax=1)
    ax[2].imshow(mask_zoom_edge, vmin=0, vmax=1)
show_masks

**Exercise** Try to determine a threshold for the DAPI nuclei.

In [ ]:
from ipywidgets import interactive
@interactive
def show_nuc_masks(nuc_thresh=(0, data.max() * 0.3, 40)):
    fig, ax = plt.subplots(1, 3, figsize=(16, 4))
    nuc_mask = np.zeros(nucleus_data.shape)
    nuc_mask[nucleus_data >= nuc_thresh] = 1
    nuc_mask_zoom_center = nuc_mask[nuc_minY1 : nuc_maxY1 , nuc_minX1 : nuc_maxX1]
    nuc_mask_zoom_edge = nuc_mask[nuc_minY2 : nuc_maxY2 , nuc_minX2 : nuc_maxX2]
    ax[0].imshow(nuc_mask, vmin=0, vmax=1)
    ax[1].imshow(nuc_mask_zoom_center, vmin=0, vmax=1)
    ax[2].imshow(nuc_mask_zoom_edge, vmin=0, vmax=1)
show_nuc_masks

### Automated detection of foreground using Otsu's method

Nobuyuki Otsu proposed a method (now very widely used) to detect thresholds. Simply put, the idea is to assume that background pixels (unwanted), and foreground pixels (your signal) will follow a bimodal distribution, i.e. that all the background pixels will be a well defined group on a histogram, which will be different from another well defined group that will be brighter, and is the signal that you want.

In [ ]:
from skimage import filters

thresh = filters.threshold_otsu(data)
print("The objective masking threshold for this dataset is:", thresh)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(16, 4))
mask = np.zeros(nodrug_slice["actin"].shape)
mask[nodrug_slice["actin"] >=thresh] = 1
mask_zoom_center = mask[minY1 : maxY1 , minX1 : maxX1]
mask_zoom_edge = mask[minY2 : maxY2 , minX2 : maxX2]
ax[0].imshow(mask, vmin=0, vmax=1)
ax[1].imshow(mask_zoom_center, vmin=0, vmax=1)
ax[2].imshow(mask_zoom_edge, vmin=0, vmax=1)

Note that the global threshold produces masks with different qualities at the edges and the center of the image because of the uneven illumination throughout the sample. Observe the histogram of pixel intensities to see why this might be the case.

In [ ]:
sns.distplot(data.flatten(), hist_kws={'log': True}, kde=False)
plt.axvline(thresh, ls='--', lw=2, c='r', label='Otsu threshold')
plt.legend()

Both manual threshold determination and Otsu's threshold determination fail to produce high-quality masks in this dataset. Noise, uneven illumination, and background, which are all common in fluorescent microscopy datasets in biology, can be corrected using a set of *rank filters*

**Exercise** View the histogram of pixel values for the "nucleus" dataset. Does Otsu's method make sense? 

In [ ]:
nuc_thresh = filters.threshold_otsu(nucleus_data)
print("The objective masking threshold for the nucleus is:", nuc_thresh)

In [ ]:
sns.distplot(nucleus_data.flatten(), hist_kws={'log': True}, kde=False)
plt.axvline(nuc_thresh, ls='--', lw=2, c='r', label='Otsu threshold')
plt.legend()

**NOTE:** we will save the application of Otsu's thresholding method to the image by imshow the mask in the next exercise.

Now back to our actin data, let's check out whether the Otsu's threshold makes sense for a cleaner image with a pre-determined mask:

In [ ]:
from skimage.io import imread
clean_mask = imread("../fig/processed_sample_mask.tif")
clean_im = imread("../fig/processed_sample_data.tif")

clean_mask = scipy.ndimage.zoom(clean_mask, mask.shape[0]/clean_mask.shape[0], order=0)
clean_im = scipy.ndimage.zoom(clean_im, mask.shape[0]/clean_im.shape[0], order=0)

fig, ax = plt.subplots(1, 3, figsize=(16, 4))
cmask_zoom_center = clean_mask[minY1 : maxY1 , minX1 : maxX1]
cmask_zoom_edge = clean_mask[minY2 : maxY2 , minX2 : maxX2]
ax[0].imshow(clean_mask, vmin=0, vmax=1)
ax[1].imshow(cmask_zoom_center, vmin=0, vmax=1)
ax[2].imshow(cmask_zoom_edge, vmin=0, vmax=1)

top = clean_im.max() * image_view_thresh
fig, ax = plt.subplots(1, 3, figsize=(16, 4))
cim_zoom_center = clean_im[minY1 : maxY1 , minX1 : maxX1]
cim_zoom_edge = clean_im[minY2 : maxY2 , minX2 : maxX2]
ax[0].imshow(clean_im, vmin=0, vmax=top)
ax[1].imshow(cim_zoom_center, vmin=0, vmax=top)
ax[2].imshow(cim_zoom_edge, vmin=0, vmax=top)

In [ ]:
from skimage import filters
threshC = filters.threshold_otsu(clean_im)

sns.distplot(clean_im.flatten(), hist_kws={'log': False}, kde=False)
plt.axvline(threshC, ls='--', lw=2, c='r')
plt.gca().set_ylim([0, 300000])

This threshold seems like it's dividing a cluster instead of separating two clusters, even though empirically the results look very good. Some algorithms become staples of images processing not because the underlying model they reflect is correct, but because they're extremely robust.

**Exercise** Determine what is incorrect about the application of Otsu's method to the nucleus data below (there are two independent errors).

In [ ]:
from skimage import filters
nuc_thresh = filters.threshold_li(nucleus_data) # Error 1: threshold method name is wrong!
print("the Otsu masking threshold for this dataset is:", nuc_thresh)

fig, ax = plt.subplots(1, 3, figsize=(16, 4))
nuc_mask = np.zeros(nucleus_data.shape)
nuc_mask[nucleus_data <= nuc_thresh] = 1 # Error 2: binary assignment should be 0

nuc_mask_zoom_center = nuc_mask[nuc_minY1 : nuc_maxY1 , nuc_minX1 : nuc_maxX1]
nuc_mask_zoom_edge = nuc_mask[nuc_minY2 : nuc_maxY2 , nuc_minX2 : nuc_maxX2]

ax[0].imshow(nuc_mask, vmin=0, vmax=1)
ax[1].imshow(nuc_mask_zoom_center, vmin=0, vmax=1)
ax[2].imshow(nuc_mask_zoom_edge, vmin=0, vmax=1)

Next: Rank filters for locally-informed image manipulations